Load Dataset

In [36]:
import timeit
start_time = timeit.default_timer()

def print_elapsed_time():
    start_time = timeit.default_timer()
    
def start_timer():
    elapsed = (timeit.default_timer() - start_time)/60
    print(elapsed)

In [19]:
from sklearn.datasets import load_files   
from keras.utils import np_utils
import numpy as np
from glob import glob




def load_dataset(data_path, shuffle=None):
    kwargs = {}
    if shuffle != None:
        kwargs['shuffle'] = shuffle
    data = load_files(data_path, **kwargs)
    img_files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 3)
    return img_files, targets



start_timer()
train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test', shuffle=False)

# load lables
label_name = [item[11:-1] for item in sorted(glob("data/train/*/"))]

print_elapsed_time()

4.838409904550645e-06


In [20]:
start_timer()

print('train_files size: {}'.format(len(train_files)))
print('train_files shape: {}'.format(train_files.shape))
print('target shape: {}'.format(train_targets.shape))
print(label_name)

start_time = timeit.default_timer()
elapsed = (timeit.default_timer() - start_time)/60
print(elapsed)


print_elapsed_time()


4.641631641528268
train_files size: 2000
train_files shape: (2000,)
target shape: (2000, 3)
['melanoma', 'nevus', 'seborrheic_keratosis']
9.130685384661774e-07


In [21]:
start_timer()
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from tqdm import tqdm


def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(384, 256))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(image_paths):
    return np.vstack([path_to_tensor(path) for path in image_paths])

print_elapsed_time()


0.0010503104177056837


Transfer Learning and putting images into tensor.

In [22]:
start_timer()

train_tensors = paths_to_tensor(tqdm(train_files))
valid_tensors = paths_to_tensor(tqdm(valid_files))
test_tensors = paths_to_tensor(tqdm(test_files))

print(train_tensors.shape)

print_elapsed_time()


0.0014982516017501741


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [05:40<00:00,  1.76it/s]


(2000, 384, 256, 3)


Flip Images

In [23]:
start_timer()


import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator


apply_train_image_transform = False

if apply_train_image_transform:
    # Caution: Doesn't guarantee prevention of duplication.
    datagen_train = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True)
    
    datagen_train.fit(train_tensors)
    shape = (train_tensors.shape[0] * 2,) + train_tensors.shape[1:]
    generated = np.ndarray(shape=shape)
    for i, image in tqdm(enumerate(train_tensors)):
        generated[i] = datagen_train.random_transform(image)
    
    train_tensors = np.concatenate((train_tensors, generated))
    train_targets = train_targets.repeat(2, axis=0)
    
print_elapsed_time()


13.792365996098898


Transfer learning using Inception Resnet V2¶

In [24]:
start_timer()

train_imgs_preprocess = preprocess_input(train_tensors)
valid_imgs_preprocess = preprocess_input(valid_tensors)
test_imgs_preprocess = preprocess_input(test_tensors)
del train_tensors, valid_tensors, test_tensors

print_elapsed_time()


13.793053152706186


In [25]:
start_timer()

from keras.applications.inception_resnet_v2 import InceptionResNetV2
transfer_model = InceptionResNetV2(include_top=False)

train_data = transfer_model.predict(train_imgs_preprocess)
valid_data = transfer_model.predict(valid_imgs_preprocess)
test_data = transfer_model.predict(test_imgs_preprocess)

del train_imgs_preprocess, valid_imgs_preprocess, test_imgs_preprocess
print(train_data.shape)

print_elapsed_time()


13.832828193098809
(2000, 10, 6, 1536)


In [26]:
start_timer()

from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential

my_model = Sequential()

my_model.add(GlobalAveragePooling2D(input_shape=train_data.shape[1:]))
my_model.add(Dropout(0.2))
my_model.add(Dense(1024, activation='relu'))
my_model.add(Dropout(0.2))
my_model.add(Dense(3, activation='softmax'))
my_model.summary()

print_elapsed_time()


67.63998663335929
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 1536)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1573888   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 3075      
Total params: 1,576,963
Trainable params: 1,576,963
Non-trainable params: 0
_________________________________________________________________


In [27]:
start_timer()

my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print_elapsed_time()


67.64254146739158


In [28]:
start_timer()

from keras.callbacks import ModelCheckpoint


checkpoint_filepath = 'saved_models/weights.best.my.hdf5'

my_checkpointer = ModelCheckpoint(filepath=checkpoint_filepath,
                               verbose=1, save_best_only=True)

my_model.fit(train_data, train_targets, 
          validation_data=(valid_data, valid_targets),
          epochs=70, batch_size=200, callbacks=[my_checkpointer], verbose=1)

print_elapsed_time()


67.64438656557036
Train on 2000 samples, validate on 150 samples
Epoch 1/70
2000/2000 [==============================] - ETA: 1:04 - loss: 1.5355 - acc: 0.155 - ETA: 29s - loss: 2.7951 - acc: 0.420 - ETA: 17s - loss: 3.3344 - acc: 0.50 - ETA: 11s - loss: 3.4588 - acc: 0.52 - ETA: 8s - loss: 3.0836 - acc: 0.5690 - ETA: 5s - loss: 2.7448 - acc: 0.584 - ETA: 3s - loss: 2.4783 - acc: 0.585 - ETA: 2s - loss: 2.3434 - acc: 0.536 - ETA: 0s - loss: 2.2224 - acc: 0.505 - 10s 5ms/step - loss: 2.1114 - acc: 0.4845 - val_loss: 0.9932 - val_acc: 0.5133

Epoch 00001: val_loss improved from inf to 0.99318, saving model to saved_models/weights.best.my.hdf5
Epoch 2/70
2000/2000 [==============================] - ETA: 1s - loss: 0.9658 - acc: 0.485 - ETA: 1s - loss: 0.8843 - acc: 0.565 - ETA: 1s - loss: 0.8492 - acc: 0.606 - ETA: 0s - loss: 0.8372 - acc: 0.630 - ETA: 0s - loss: 0.8285 - acc: 0.645 - ETA: 0s - loss: 0.8427 - acc: 0.649 - ETA: 0s - loss: 0.8328 - acc: 0.657 - ETA: 0s - loss: 0.8443 - acc:

2000/2000 [==============================] - ETA: 1s - loss: 0.5522 - acc: 0.825 - ETA: 1s - loss: 0.5875 - acc: 0.772 - ETA: 1s - loss: 0.5949 - acc: 0.768 - ETA: 0s - loss: 0.6026 - acc: 0.756 - ETA: 0s - loss: 0.5869 - acc: 0.755 - ETA: 0s - loss: 0.5721 - acc: 0.762 - ETA: 0s - loss: 0.5731 - acc: 0.758 - ETA: 0s - loss: 0.5774 - acc: 0.756 - ETA: 0s - loss: 0.5680 - acc: 0.763 - 2s 773us/step - loss: 0.5693 - acc: 0.7645 - val_loss: 0.7322 - val_acc: 0.7133

Epoch 00015: val_loss did not improve from 0.72248
Epoch 16/70
2000/2000 [==============================] - ETA: 1s - loss: 0.5536 - acc: 0.785 - ETA: 1s - loss: 0.5412 - acc: 0.790 - ETA: 1s - loss: 0.5841 - acc: 0.760 - ETA: 0s - loss: 0.6198 - acc: 0.736 - ETA: 0s - loss: 0.5950 - acc: 0.748 - ETA: 0s - loss: 0.5885 - acc: 0.755 - ETA: 0s - loss: 0.5913 - acc: 0.752 - ETA: 0s - loss: 0.5904 - acc: 0.752 - ETA: 0s - loss: 0.5784 - acc: 0.762 - 2s 754us/step - loss: 0.5750 - acc: 0.7605 - val_loss: 0.8023 - val_acc: 0.6667

E

Epoch 00029: val_loss improved from 0.67879 to 0.66892, saving model to saved_models/weights.best.my.hdf5
Epoch 30/70
2000/2000 [==============================] - ETA: 1s - loss: 0.4953 - acc: 0.805 - ETA: 1s - loss: 0.4697 - acc: 0.817 - ETA: 0s - loss: 0.4844 - acc: 0.815 - ETA: 0s - loss: 0.4906 - acc: 0.803 - ETA: 0s - loss: 0.4842 - acc: 0.803 - ETA: 0s - loss: 0.4792 - acc: 0.807 - ETA: 0s - loss: 0.4786 - acc: 0.803 - ETA: 0s - loss: 0.4787 - acc: 0.802 - ETA: 0s - loss: 0.4745 - acc: 0.803 - 2s 766us/step - loss: 0.4725 - acc: 0.8050 - val_loss: 0.7247 - val_acc: 0.7067

Epoch 00030: val_loss did not improve from 0.66892
Epoch 31/70
2000/2000 [==============================] - ETA: 1s - loss: 0.4286 - acc: 0.815 - ETA: 1s - loss: 0.4359 - acc: 0.815 - ETA: 0s - loss: 0.4500 - acc: 0.805 - ETA: 0s - loss: 0.4462 - acc: 0.813 - ETA: 0s - loss: 0.4559 - acc: 0.808 - ETA: 0s - loss: 0.4579 - acc: 0.812 - ETA: 0s - loss: 0.4567 - acc: 0.813 - ETA: 0s - loss: 0.4549 - acc: 0.812 - ET

Epoch 00044: val_loss did not improve from 0.65706
Epoch 45/70
2000/2000 [==============================] - ETA: 1s - loss: 0.4294 - acc: 0.810 - ETA: 1s - loss: 0.4183 - acc: 0.807 - ETA: 1s - loss: 0.3940 - acc: 0.828 - ETA: 0s - loss: 0.4030 - acc: 0.831 - ETA: 0s - loss: 0.4046 - acc: 0.834 - ETA: 0s - loss: 0.4086 - acc: 0.830 - ETA: 0s - loss: 0.4084 - acc: 0.832 - ETA: 0s - loss: 0.4025 - acc: 0.839 - ETA: 0s - loss: 0.3996 - acc: 0.840 - 2s 768us/step - loss: 0.3974 - acc: 0.8445 - val_loss: 0.7115 - val_acc: 0.7067

Epoch 00045: val_loss did not improve from 0.65706
Epoch 46/70
2000/2000 [==============================] - ETA: 1s - loss: 0.4059 - acc: 0.825 - ETA: 1s - loss: 0.3868 - acc: 0.840 - ETA: 0s - loss: 0.3806 - acc: 0.845 - ETA: 0s - loss: 0.3909 - acc: 0.842 - ETA: 0s - loss: 0.3854 - acc: 0.847 - ETA: 0s - loss: 0.3881 - acc: 0.843 - ETA: 0s - loss: 0.3911 - acc: 0.836 - ETA: 0s - loss: 0.4002 - acc: 0.829 - ETA: 0s - loss: 0.3916 - acc: 0.833 - 2s 812us/step - los

2000/2000 [==============================] - ETA: 1s - loss: 0.3398 - acc: 0.850 - ETA: 1s - loss: 0.3202 - acc: 0.870 - ETA: 0s - loss: 0.3231 - acc: 0.876 - ETA: 0s - loss: 0.3330 - acc: 0.867 - ETA: 0s - loss: 0.3345 - acc: 0.869 - ETA: 0s - loss: 0.3304 - acc: 0.872 - ETA: 0s - loss: 0.3254 - acc: 0.876 - ETA: 0s - loss: 0.3271 - acc: 0.876 - ETA: 0s - loss: 0.3221 - acc: 0.881 - 2s 753us/step - loss: 0.3253 - acc: 0.8780 - val_loss: 0.7571 - val_acc: 0.7067

Epoch 00060: val_loss did not improve from 0.63663
Epoch 61/70
2000/2000 [==============================] - ETA: 1s - loss: 0.3484 - acc: 0.855 - ETA: 1s - loss: 0.2999 - acc: 0.870 - ETA: 0s - loss: 0.2886 - acc: 0.885 - ETA: 0s - loss: 0.3004 - acc: 0.883 - ETA: 0s - loss: 0.3158 - acc: 0.877 - ETA: 0s - loss: 0.3233 - acc: 0.875 - ETA: 0s - loss: 0.3292 - acc: 0.872 - ETA: 0s - loss: 0.3279 - acc: 0.875 - ETA: 0s - loss: 0.3291 - acc: 0.875 - 2s 758us/step - loss: 0.3264 - acc: 0.8755 - val_loss: 0.7871 - val_acc: 0.7400

E

In [29]:
start_timer()

my_model.load_weights(checkpoint_filepath)

print_elapsed_time()


69.74688307414986


Evaluavate 

In [30]:
start_timer()

import csv


my_predictions = [my_model.predict(np.expand_dims(feature, axis=0)) for feature in test_data]

# test_accuracy = 100 * np.sum(np.array(my_predictions)==np.argmax(test_targets, axis=1)) / len(my_predictions)
# print('Test accuracy: %.4f%%' % test_accuracy)

with open('my_transfer.csv', 'w', newline='') as csvfile:
    result_writger = csv.writer(csvfile)
    result_writger.writerow(['Id', 'task_1', 'task_2'])
    for test_filepath, test_prediction in zip(test_files, my_predictions):
        result_writger.writerow([test_filepath, test_prediction[0][0], test_prediction[0][2]])
        
print_elapsed_time()


69.79439494532085


Category 1 Score: 0.526
Category 2 Score: 0.606
Category 3 Score: 0.566

![ROC Curves](images/figure_1.png)

Without transfer learning, loading images into tensor

In [35]:
start_timer()
train_tensors = paths_to_tensor(tqdm(train_files))
train_tensors = train_tensors.astype('float32') / 255

valid_tensors = paths_to_tensor(tqdm(valid_files))
valid_tensors = valid_tensors.astype('float32') / 255

test_tensors = paths_to_tensor(tqdm(test_files))
test_tensors = test_tensors.astype('float32') / 255

print(train_tensors.shape)

print_elapsed_time()

0.10808795256416488


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [06:12<00:00,  1.61it/s]


(2000, 384, 256, 3)


In [37]:
start_timer()
from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential

my_model = Sequential()

my_model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', 
                        input_shape=train_tensors.shape[1:]))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.2))

my_model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.2))

my_model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.2))

my_model.add(Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.1))

my_model.add(Conv2D(filters=2048, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.1))


my_model.add(GlobalAveragePooling2D())

my_model.add(Dense(3, activation='softmax'))

my_model.summary()

print_elapsed_time()

5.867575357594311
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_411 (Conv2D)          (None, 384, 256, 16)      448       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 192, 128, 16)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 192, 128, 16)      0         
_________________________________________________________________
conv2d_412 (Conv2D)          (None, 192, 128, 64)      9280      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 96, 64, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 96, 64, 64)        0         
_________________________________________________________________
conv2d_413 (Conv2D)          (None, 96, 64, 256)       147

In [38]:
start_timer()
my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print_elapsed_time()

5.94980908334231


In [ ]:
start_timer()


from keras.callbacks import ModelCheckpoint
import os


checkpoint_filepath = 'saved_models/weights.best.my.hdf5'

my_checkpointer = ModelCheckpoint(filepath=checkpoint_filepath,
                               verbose=1, save_best_only=True)

my_model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=40, batch_size=100, callbacks=[my_checkpointer], verbose=1)


print_elapsed_time()

6.022846783856888
Train on 2000 samples, validate on 150 samples
Epoch 1/40
1400/2000 [====================>.........] - ETA: 31:36 - loss: 1.1120 - acc: 0.17 - ETA: 27:51 - loss: 3.1349 - acc: 0.42 - ETA: 25:58 - loss: 4.1315 - acc: 0.49 - ETA: 23:48 - loss: 4.3881 - acc: 0.53 - ETA: 21:55 - loss: 4.4131 - acc: 0.57 - ETA: 20:14 - loss: 4.4835 - acc: 0.59 - ETA: 18:39 - loss: 4.5798 - acc: 0.60 - ETA: 17:08 - loss: 4.6722 - acc: 0.61 - ETA: 15:38 - loss: 4.8336 - acc: 0.61 - ETA: 14:12 - loss: 4.7854 - acc: 0.62 - ETA: 12:47 - loss: 4.8779 - acc: 0.62 - ETA: 11:20 - loss: 4.8475 - acc: 0.63 - ETA: 9:55 - loss: 4.7846 - acc: 0.6446 - ETA: 8:29 - loss: 4.7997 - acc: 0.6479

In [ ]:
start_timer()

my_model.load_weights(checkpoint_filepath)

print_elapsed_time()

In [ ]:
start_timer()


import csv


my_predictions = [my_model.predict(np.expand_dims(feature, axis=0)) for feature in test_tensors]

# test_accuracy = 100 * np.sum(np.array(my_predictions)==np.argmax(test_targets, axis=1)) / len(my_predictions)
# print('Test accuracy: %.4f%%' % test_accuracy)

with open('my_cnn.csv', 'w', newline='') as csvfile:
    result_writger = csv.writer(csvfile)
    result_writger.writerow(['Id', 'task_1', 'task_2'])
    for test_filepath, test_prediction in zip(test_files, my_predictions):
        result_writger.writerow([test_filepath, test_prediction[0][0], test_prediction[0][2]])
        

print_elapsed_time()